# (4) Bayes repeated measures anova
 based on Rouder et al. 2012

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import bambi as bmb

In [ ]:
# model has main effects of kappa, distractor. interaction of kappa and distractor. random effect of subject.
model = bmb.Model(
    '''
    circ_sd ~ kappa*distractor  
                        + (1|subject)
    ''',
    dfV11_circstats, family='vonmises') 
# family = von mises? because we are predicting circular variable?
# family = gaussian? because circ_sd is normal?
model.build();
# idata_kwargs={"log_likelihood": True}

In [ ]:
#model.graph # group specific means random effects

In [ ]:
#results = model.fit();

In [ ]:
model.plot_priors();
#az.plot_trace(results, compact=False);
#az.summary(results)
#list(results.posterior.data_vars)
#model.backend.model

In [ ]:
# try another model following radon example
unpooled_priors = {
    "kappa:distractor": bmb.Prior("Normal", mu=0, sigma=10),
    "sigma": bmb.Prior("Normal", mu=0, sigma = 10),
}

unpooled_model = bmb.Model("circ_sd ~ 0 + kappa:distractor", dfV11_circstats, priors=unpooled_priors)
unpooled_model



In [ ]:
unpooled_results = unpooled_model.fit()

In [ ]:
az.plot_trace(data=unpooled_results, compact=True, chain_prop={"ls": "-"})
plt.suptitle("Un-Pooled Model Trace");

In [ ]:
az.plot_forest(data=unpooled_results, figsize=(2,4), r_hat=True, combined=True, textsize=8);

In [ ]:
dfV11_circstats['kappa'].replace(['low', 'high'], [0,1], inplace=True)
dfV11_circstats['distractor'].replace(['none', 'ignore', 'attend'], [0,1,2], inplace=True)
dfV11_circstats['subject'].astype('int')
dfV11_circstats.head()

In [ ]:
mu_circ =   beta_d * dfV11_circstats['distractor'] 
mu_circ.shape
(dfV11_circstats['circ_sd']).shape

In [ ]:
# try calculating individual BF10s for each model
# model1: circ_sd ~ kappa +(subject)
# hierarchical circular regression
# change kappa to 1 or 2
# change subject to numerical

models = [];
idatas = [];
with pm.Model() as model:
    # Priors for model parameters
    beta = pm.Normal('beta', mu=0, sigma=10, shape=(1,1))  # Coefficients for categorical predictor
    beta_d = pm.Normal('beta_d', mu=0, sigma=10, shape=(1,1)) 
    intercept = pm.Normal('intercept', mu=0, sigma=10)  # Intercept noise not accounted for by kappa and subject
    # Random effects for subjects
    subject_beta = pm.Normal('subject_beta', mu=0, sigma=10, shape=(1,1))
    
    mu_circ = beta * dfV11_circstats['kappa'] + beta_d * dfV11_circstats['distractor'] + subject_beta * dfV11_circstats['subject'] + intercept
    # Likelihood   
    circ_est = pm.Normal('circ_outcome', mu=mu_circ, sigma = 10, observed= np.array(dfV11_circstats['circ_sd']).reshape((1,144)))

 
    trace = pm.sample(1000, tune=1000)
    #idata_circ = pm.sample_smc(random_seed=42)
    #idata_circ.extend(pm.sample_prior_predictive(8000))
    #models.append(model)
    #idatas.append(idata_circ)

In [ ]:
with model:
    az.plot_trace(trace)

In [ ]:
with model:
    display(az.summary(trace, round_to=2))

In [ ]:
BF_smc = np.exp(
    idatas[1].sample_stats["log_marginal_likelihood"].mean()
    - idatas[0].sample_stats["log_marginal_likelihood"].mean()
)
np.round(BF_smc).item()

In [ ]:
# this might be the answer to my problems:
#https://discourse.pymc.io/t/hierarchical-regression-models-for-ratings-data-2-by-2-within-subject-design/4206
subjs = 24
with pm.Model() as example_LMM:
    
    # priors
    mu_a = pm.Normal('mua_a', mu=0., sigma=10)
    sigma_a = pm.Exponential('sigma_a', 5)
    mu_b = pm.Normal('mu_b', mu=0., sigma=10)
    sigma_b = pm.Exponential('sigma_b', 5)
    
    # Random intecepts
    a = pm.Normal('a', mu=mu_a, sigma=sigma_a, shape=subjs)
    b1 = pm.Normal('b1', mu=mu_b, sigma=sigma_b, shape=subjs)
    b2 = pm.Normal('b2', mu=mu_b, sigma=sigma_b, shape=subjs)
    
    # Model error
    kappa = pm.Uniform('kappa', lower = 0, upper=100)
    
    # expected value
    y_hat = a[subj] + b1[subj]*blockType + b2[subj]*CSType
    
    # Data likelihood
    y_like = pm.Normal('y_like', mu=y_hat, kappa=kappa, observed=dfV11_circstats['circ_sd'])
    
model_to_graphviz(example_LMM)

In [ ]:
with example_LMM:
    prior_checks = pm.sample_prior_predictive(random_seed=RANDOM_SEED)